In [11]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import easyocr

In [12]:
def TextDetection(platepath : str) :
	reader = easyocr.Reader(['en'])
	img = cv2.imread(platepath)
	results = reader.readtext(img)
	plate_number = ""
	for res in results:
		letter = res[1]
		plate_number += letter
	return plate_number

In [13]:
def ObjectFinder(bin_img, real_img) :
	objects_list = cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
	parents_idx_list = [objects_list[1][0][i][3] for i in range(0, len(objects_list[1][0])) if objects_list[1][0][i][3] != -1]
	parents_idx_list = np.unique(np.array(parents_idx_list)).tolist()
	counter = 0
	path_list = []
	for idx in parents_idx_list :
		perimeter = cv2.arcLength(objects_list[0][idx], True)
		polygon = cv2.approxPolyDP(objects_list[0][idx], 10, True)
		if len(polygon) == 4 :
			mask1 = np.zeros(bin_img.shape, np.uint8)
			cv2.fillConvexPoly(mask1, objects_list[0][idx], 255)
			x, y, w, h = cv2.boundingRect(objects_list[0][idx])
			cropped = cv2.bitwise_and(real_img, real_img, mask = mask1)[y:y+h, x:x+w]
			cv2.imwrite("cropped_results/" + str(counter) + ".png", cropped)
			path_list.append("cropped_results/" + str(counter) + ".png")
			counter += 1
	return path_list

In [14]:
def Method1(gray_img) :
	gray_img_filtered = cv2.bilateralFilter(gray_img, -1, 20, 20)
	threshold_from_otsu, bin_tozero_img = cv2.threshold(gray_img_filtered, 0, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)
	return cv2.adaptiveThreshold(bin_tozero_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 4)

In [15]:
def Method2(gray_img) :
	gray_img_filtered = cv2.bilateralFilter(gray_img, -1, 20, 20)
	threshold_from_otsu, bin_tozero_img = cv2.threshold(gray_img_filtered, 0, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)
	return cv2.Canny(gray_img_filtered, threshold_from_otsu / 2, threshold_from_otsu)

In [16]:
def Method3(gray_img) :
	gray_img_filtered = cv2.bilateralFilter(gray_img, -1, 20, 20)
	return cv2.Laplacian(gray_img_filtered, -1)

In [17]:
def PlateDetection(imagepath : str, Method) :
	img = cv2.imread(imagepath)
	result = Method(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))
	path_list = ObjectFinder(result, img)
	for path in path_list :
		plate_number = TextDetection(path)
		if len(plate_number) != 0 :
			return plate_number
	return ""
	

In [18]:
# for i in range(14):
# 	result = PlateDetection("images/" + str(i) + ".png", Method1)
# 	if len(result) == 0 :
# 		result = PlateDetection("images/" + str(i) + ".png", Method2)
# 	if len(result) == 0 :
# 		result = PlateDetection("images/" + str(i) + ".png", Method3)

# 	print(result)

In [1]:
from Detection import *

import os
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog

def LoadImage(filepath : str):
	image = PhotoImage(file=filepath, master=imgframe)
	loaded_image = ttk.Label(imgframe)
	loaded_image.configure(image=image)
	loaded_image.image = image
	loaded_image.grid(column=0, row=0, sticky="nsew")

def PlateToText(filename : str):
	result = PlateDetection(filename, Method1)
	if len(result) == 0 :
		result = PlateDetection(filename, Method2)
	if len(result) == 0 :
		result = PlateDetection(filename, Method3)
	if len(result) == 0 :
		result = "nie znaleziono"
	output = tk.Label(menuframe).grid(column=1, row=1)
	output['text'] = result

def browseFiles():
	filename = tk.filedialog.askopenfilename(initialdir = os.getcwd(),
										title = "Select an Image",
										filetypes = [("Images", "*.png")])
	LoadImage(filename)
	PlateToText(filename)

root = Tk()
root.title("Licence Plate Recognition")
root.resizable(False, False)
root.geometry("800x600")

mainframe = ttk.Frame(root)
mainframe.grid()
imgframe = ttk.Frame(mainframe, borderwidth=1, relief="groove", width=800, height=480)
imgframe.grid(column=0, row=0)
imgframe.grid_rowconfigure(0, weight=1)
imgframe.grid_columnconfigure(0, weight=1)
imgframe.grid_propagate(False)

spacing = ttk.Frame(mainframe, width=800, height=20).grid(column=0, row=1)

menuframe = ttk.Frame(mainframe, borderwidth=1, relief="groove", width=800, height=100, padding="3 3 3 3")
menuframe.grid(column=0, row=2)
# menuframe.grid_rowconfigure(0, weight=1)
menuframe.grid_columnconfigure(1, weight=1)
menuframe.grid_propagate(False)

tk.Button(menuframe, text="Load image", height=7, command=browseFiles).grid(column=0, row=0, rowspan = 2)
plate_label = tk.Label(menuframe, text='Recognised plate number:').grid(column=1, row=0)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/jakub/anaconda3/envs/pite/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_358/65765268.py", line 30, in browseFiles
    PlateToText(filename)
  File "/tmp/ipykernel_358/65765268.py", line 15, in PlateToText
    result = PlateDetection(filename, Method1)
NameError: name 'PlateDetection' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/jakub/anaconda3/envs/pite/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_358/65765268.py", line 30, in browseFiles
    PlateToText(filename)
  File "/tmp/ipykernel_358/65765268.py", line 15, in PlateToText
    result = PlateDetection(filename, Method1)
NameError: name 'PlateDetection' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/jakub/anaconda3/envs/pite/lib/pytho